In [35]:
import torch 
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [36]:
data = pd.read_csv("../csv_building_damage_assessment.csv",na_values = ["None"," ","nan","\n"])
data = data.drop(["building_id","district_id","vdcmun_id","ward_id"],axis = 1)

for column in data.columns:
    if data[column].count() <300000:
        data = data.drop(column,axis = 1)

data = data.dropna()
label = data.pop("damage_grade")
label = pd.get_dummies(label)
label = np.array(label)

features = pd.get_dummies(data)

features = torch.tensor(np.array(features),dtype = torch.float32)
label = torch.tensor(label,dtype = torch.float32)

print(features.shape)

torch.Size([199041, 24])


In [37]:
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 1000
dataset = TensorDataset(features[:199000], label[:199000])
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [38]:
model = torch.nn.Sequential(
    torch.nn.Linear(24,60),
    torch.nn.ReLU(),
    torch.nn.Linear(60,30),
    torch.nn.ReLU(),
    torch.nn.Linear(30,15),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(15,5),
    torch.nn.Softmax(dim=-1)
)

In [39]:
loss_fn = torch.nn.BCELoss()

In [40]:
opt = torch.optim.Adam(model.parameters(),lr=1e-4)

In [42]:
from torch import no_grad
epochs = 1000
for epoch in range(epochs):
    for features,labels in dataloader:
        # print(features.shape, labels.shape)
        preds = model(features)
        loss = loss_fn(preds,labels)

        loss.backward()
        opt.step()
        opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for features,labels in dataloader:
                preds = model(features)
                for i in range(len(preds)):
                    for j in range(len(preds[i])):
                            
                            if preds[i][j]==max(preds[i]):
                                if(label[i][j]==1):
                                    right+=1

            print("Accuracy :",round(right*100/(len(dataloader)*BATCH_SIZE),5))

Accuracy : 22.42111
Accuracy : 22.34724
Accuracy : 22.4995


In [ ]:
torch.save(model.state_dict(),"earthquakedamage.pth")